In [4]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np


df = pd.read_csv("PATH_OF_DATASET_HERE")
df.head()

,Text,Physics,Chemistry,Maths
0,"The Cartoon Guide to Chemistry, Larry Gonick",0,1,0
1,Elemental: How the Periodic Table Can Now Expl...,0,1,0
2,A Mathematician's Lament: How School Cheats Us...,0,0,1
3,"the right stuff, tom wolfe",1,0,0
4,"introduction to electrodynamics, david j. grif...",1,0,0


USING TEXT VECTORIZATION TO CREATE A TOKEN DICTIONARY

In [6]:
from tensorflow.keras.layers import TextVectorization
x = df["Text"]
y = df[df.columns[1:]].values

In [7]:
max_words = 50000
vectorizer = TextVectorization(max_tokens = max_words, output_sequence_length = 100, output_mode = "int")

vectorizer.adapt(x.values)

In [8]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'of',
 'a',
 'and',
 'to',
 'physics',
 'in',
 'science',
 'mathematical',
 'how',
 'history',
 'goodreads',
 'chemistry',
 'author',
 'mathematics',
 'richard',
 'for',
 'universe',
 'from',
 'david',
 'quantum',
 'new',
 'math',
 'feynman',
 'story',
 'stephen',
 'time',
 'paul',
 'on',
 'genius',
 'world',
 'steven',
 'p',
 'john',
 'j',
 'introduction',
 'hawking',
 'that',
 'stewart',
 's',
 'reality',
 'man',
 'ian',
 'book',
 'an',
 'with',
 'william',
 'who',
 'martin',
 'm',
 'life',
 'is',
 'infinity',
 'everything',
 'brief',
 'through',
 'theory',
 'simon',
 'scientific',
 'robert',
 'our',
 'one',
 'mechanics',
 'h',
 'elements',
 'changed',
 'at',
 'worlds',
 'what',
 'us',
 'things',
 'strogatz',
 'search',
 'roger',
 'r',
 'penrose',
 'numbers',
 'number',
 'molecules',
 'modern',
 'love',
 'leonard',
 'james',
 'hidden',
 'greatest',
 'd',
 'chemical',
 'carroll',
 'c',
 'brian',
 'birth',
 'big',
 'why',
 'w',
 'two',
 'truth',
 'true',
 'tim',

In [9]:
vectorized_text = vectorizer(x.values)
vectorized_text

<tf.Tensor: shape=(259, 100), dtype=int64, numpy=
array([[  2, 941, 240, ...,   0,   0,   0],
       [838,  11,   2, ...,   0,   0,   0],
       [  4, 211, 656, ...,   0,   0,   0],
       ...,
       [  2, 126, 189, ...,   0,   0,   0],
       [658,   3, 483, ...,   0,   0,   0],
       [  4, 153,   3, ...,   0,   0,   0]], dtype=int64)>

DIVIDING THE DATASET INTO BATCHES

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(42)
dataset = dataset.batch(18)
dataset = dataset.prefetch(8)

In [11]:
batch_x, batch_y = dataset.as_numpy_iterator().next()

In [12]:
int(len(dataset))

15

In [13]:
train = dataset.take(int(len(dataset)*.8))
val = dataset.skip(int(len(dataset)*.8)).take(int(len(dataset)*.1))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [14]:
len(test)

1

In [15]:
train_generator = train.as_numpy_iterator()
train_generator.next()

(array([[313, 409,   6, ...,   0,   0,   0],
        [270,   3,   4, ...,   0,   0,   0],
        [861, 126, 521, ...,   0,   0,   0],
        ...,
        [ 55,   5, 202, ...,   0,   0,   0],
        [337,   3,  24, ...,   0,   0,   0],
        [121,   4, 148, ...,   0,   0,   0]], dtype=int64),
 array([[1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0]], dtype=int64))

MODEL CREATION

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [17]:
model = Sequential()

model.add(Embedding(max_words + 1, 32))

model.add(Bidirectional(LSTM(32, activation = "tanh")))

model.add(Dense(128, activation = "relu" ))
model.add(Dense(256, activation = "relu" ))
model.add(Dense(128, activation = "relu" ))

model.add(Dense(3, activation = "sigmoid" ))

In [18]:
model.compile(loss = "BinaryCrossentropy", optimizer = "Adam")

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          1600032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                        

TRAINING

In [20]:
history = model.fit(train, epochs = 30, validation_data = val)

Epoch 1/30


KeyboardInterrupt: 

In [21]:
from matplotlib import pyplot as plt
pd.DataFrame(history.history).plot()
plt.show()

NameError: name 'history' is not defined

In [ ]:
batch = test.as_numpy_iterator().next()
input_text = vectorizer("plane trignometry")
model.predict(np.expand_dims(input_text, 0))

1/1 [==============================] - 0s 485ms/step


array([[8.9993548e-01, 9.8624194e-05, 8.0265187e-02]], dtype=float32)

In [ ]:
batch_x, batch_y = test.as_numpy_iterator().next()
batch_y

array([[0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0]], dtype=int64)

TESTING

In [ ]:
model.predict(batch_x)

1/1 [==============================] - 0s 16ms/step


array([[1.4663765e-05, 1.0543269e-03, 9.9861240e-01],
       [1.3585849e-04, 3.5456292e-04, 9.9923778e-01],
       [2.0232362e-06, 9.9985731e-01, 2.6752925e-04],
       [1.2845477e-06, 9.9990857e-01, 1.6436762e-04],
       [1.7855715e-03, 1.1121560e-03, 9.9560320e-01],
       [9.9999535e-01, 1.6809247e-08, 2.4936987e-06],
       [9.9900836e-01, 9.9628460e-06, 7.6758419e-04],
       [2.9607423e-04, 6.9866903e-05, 9.9952602e-01],
       [8.7882324e-07, 9.9989438e-01, 2.4390215e-04],
       [2.3177240e-07, 9.9997318e-01, 5.3413951e-05],
       [2.9501619e-04, 1.0859162e-04, 9.9943560e-01],
       [9.1030952e-06, 9.9926013e-01, 1.5368931e-03],
       [8.1637586e-03, 3.7958074e-05, 9.9161667e-01],
       [1.9665461e-04, 1.7051956e-03, 9.9686915e-01],
       [3.0939901e-04, 3.9875173e-04, 9.9886984e-01],
       [1.8860259e-04, 4.6725007e-04, 9.9897766e-01],
       [2.5351427e-03, 3.5603635e-04, 9.9647731e-01],
       [9.9984682e-01, 8.8711840e-07, 1.1332663e-04]], dtype=float32)

MEASURING THE PARAMETERS

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    x_true, y_true = batch

    yhat = model.predict(x_true)

    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

    print(f'Precision: {float(pre.result())}, Recall: {float(re.result())}, Accuracy: {float(acc.result())}')

1/1 [==============================] - 0s 16ms/step
Precision: 1.0, Recall: 1.0, Accuracy: 0.0


SAVE THE MODEL AS AN .H5 FILE

In [ ]:
model.save("genreclassifier.h5")

LOADING UP THE SAVED MODEL

In [23]:
model = tf.keras.models.load_model("genreclassifier.h5")
input_str = vectorizer("Verma")
res = model.predict(np.expand_dims(input_str, 0))
Genre = 0
l = []
for idx, col in enumerate(df.columns[1:]):
    l.append(res[0][idx])
print(l)
m = max(l)
subjects = ["Physics", "Chemistry", "Maths"]
print(subjects[l.index(m)]) 

1/1 [==============================] - 1s 523ms/step
[0.9906069, 4.25231e-06, 0.008241871]
Physics
